In [1]:
from google.colab import drive

# This will prompt you to authorize access to your Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/drive/MyDrive/dataset_action_split/train')

label_types = os.listdir('/content/drive/MyDrive/dataset_action_split/train')
print (label_types)

['Stand up', 'Sit down']


In [3]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/dataset_action_split/train' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/dataset_action_split/train' + '/' +item) + '/' + room))

# Build a dataframe
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

        tag                                         video_name
0  Stand up  /content/drive/MyDrive/dataset_action_split/tr...
1  Stand up  /content/drive/MyDrive/dataset_action_split/tr...
2  Stand up  /content/drive/MyDrive/dataset_action_split/tr...
3  Stand up  /content/drive/MyDrive/dataset_action_split/tr...
4  Stand up  /content/drive/MyDrive/dataset_action_split/tr...
          tag                                         video_name
358  Sit down  /content/drive/MyDrive/dataset_action_split/tr...
359  Sit down  /content/drive/MyDrive/dataset_action_split/tr...
360  Sit down  /content/drive/MyDrive/dataset_action_split/tr...
361  Sit down  /content/drive/MyDrive/dataset_action_split/tr...
362  Sit down  /content/drive/MyDrive/dataset_action_split/tr...


In [4]:
train_df.head()

,tag,video_name
0,Stand up,/content/drive/MyDrive/dataset_action_split/tr...
1,Stand up,/content/drive/MyDrive/dataset_action_split/tr...
2,Stand up,/content/drive/MyDrive/dataset_action_split/tr...
3,Stand up,/content/drive/MyDrive/dataset_action_split/tr...
4,Stand up,/content/drive/MyDrive/dataset_action_split/tr...


In [5]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

In [6]:
dataset_path = os.listdir('/content/drive/MyDrive/dataset_action_split/test')
print(dataset_path)

room_types = os.listdir('/content/drive/MyDrive/dataset_action_split/test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/dataset_action_split/test' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/dataset_action_split/test' + '/' +item) + '/' + room))

# Build a dataframe
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')

['Stand up', 'Sit down']
Types of activities found:  2
        tag                                         video_name
0  Stand up  /content/drive/MyDrive/dataset_action_split/te...
1  Stand up  /content/drive/MyDrive/dataset_action_split/te...
2  Stand up  /content/drive/MyDrive/dataset_action_split/te...
3  Stand up  /content/drive/MyDrive/dataset_action_split/te...
4  Stand up  /content/drive/MyDrive/dataset_action_split/te...
          tag                                         video_name
104  Sit down  /content/drive/MyDrive/dataset_action_split/te...
105  Sit down  /content/drive/MyDrive/dataset_action_split/te...
106  Sit down  /content/drive/MyDrive/dataset_action_split/te...
107  Sit down  /content/drive/MyDrive/dataset_action_split/te...
108  Sit down  /content/drive/MyDrive/dataset_action_split/te...


In [7]:
!pip install tensorflow_docs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.5/182.5 kB 3.9 MB/s eta 0:00:00


In [8]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [9]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

In [10]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 363
Total videos for testing: 109


,Unnamed: 0,video_name,tag
147,147,/content/drive/MyDrive/dataset_action_split/tr...,Stand up
196,196,/content/drive/MyDrive/dataset_action_split/tr...,Stand up
251,251,/content/drive/MyDrive/dataset_action_split/tr...,Sit down
162,162,/content/drive/MyDrive/dataset_action_split/tr...,Stand up
332,332,/content/drive/MyDrive/dataset_action_split/tr...,Sit down
236,236,/content/drive/MyDrive/dataset_action_split/tr...,Sit down
272,272,/content/drive/MyDrive/dataset_action_split/tr...,Sit down
113,113,/content/drive/MyDrive/dataset_action_split/tr...,Stand up
52,52,/content/drive/MyDrive/dataset_action_split/tr...,Stand up
96,96,/content/drive/MyDrive/dataset_action_split/tr...,Stand up


In [11]:
train_df.shape

(363, 3)

In [12]:
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [13]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87910968/87910968 [==============================] - 0s 0us/step


In [14]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['Sit down', 'Stand up']


array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [18]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [19]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()

    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values

    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 31ms/step
Frame features in train set: (363, 20, 2048)
Frame masks in train set: (363, 20)
train_labels in train set: (363, 1)
test_labels in train set: (109, 1)


In [21]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
5/8 [=================>............] - ETA: 0s - loss: 0.4433 - accuracy: 0.8313  
Epoch 1: val_loss improved from inf to 2.04323, saving model to ./tmp/video_classifier
8/8 [==============================] - 11s 359ms/step - loss: 0.3775 - accuracy: 0.8780 - val_loss: 2.0432 - val_accuracy: 0.0000e+00
Epoch 2/30
5/8 [=================>............] - ETA: 0s - loss: 0.3073 - accuracy: 0.9312
Epoch 2: val_loss did not improve from 2.04323
8/8 [==============================] - 0s 27ms/step - loss: 0.3021 - accuracy: 0.9291 - val_loss: 2.2340 - val_accuracy: 0.0000e+00
Epoch 3/30
5/8 [=================>............] - ETA: 0s - loss: 0.2726 - accuracy: 0.9312
Epoch 3: val_loss did not improve from 2.04323
8/8 [==============================] - 0s 20ms/step - loss: 0.2827 - accuracy: 0.9291 - val_loss: 2.3793 - val_accuracy: 0.0000e+00
Epoch 4/30
6/8 [=====================>........] - ETA: 0s - loss: 0.2719 - accuracy: 0.9323
Epoch 4: val_loss did not improve from 2.04323
8/8 

In [23]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: /content/drive/MyDrive/dataset_action_split/test/Sit down/video_52.avi
1/1 [==============================] - 0s 19ms/step
  Stand up: 87.59%
  Sit down: 12.41%
